Once the normalized reaction abundances are computed, combine them with the processed metadata to obtain the input dataset which will be used to fit the models.

Set the boolean ```save_files``` to ```True``` to overwrite the files.

In [ ]:
save_files = False

In [ ]:
import pandas as pd

In [ ]:
original_abund = "../data/processed_files/normalized_reaction_abundance.csv"
original_metadata = "../data/processed_files/metadata.csv"

In [ ]:
orig_reac = pd.read_csv(original_abund)
orig_reac = orig_reac[[c for c in orig_reac.columns if "abund_rxns" in c]]

orig_rxn = pd.read_csv(original_abund)
orig_rxn = orig_rxn[[c for c in orig_rxn.columns if "abund_rxns" not in c]]

orig_meta = pd.read_csv(original_metadata)

In [ ]:
# Label the reactions by their name

orig_reac.index = orig_rxn["name"]
orig_reac.columns = orig_meta["#SampleID"]
orig_full = pd.merge(
    orig_reac.transpose(), orig_meta, left_index=True, right_on="#SampleID"
)
orig_full.index = orig_full["#SampleID"]

In [ ]:
orig_full

In [ ]:
# Save the data
rxns = [c for c in orig_full.columns if c not in orig_meta.columns]
if save_files:
    orig_full.to_csv("../data/processed_files/abund_and_metadata.csv")
    pd.DataFrame(rxns).to_csv("../data/processed_files/rxn_names.csv")

# A small sanity check

We will only process the samples from Wild, SPF, and human samples with known gender. It is possible, that some reactions would have 0 abundance for all samples due to the removal of certain models. This is very unlikely as we only removed defined microbiomes from the analysis, but we will run a sanity check to be sure. 

In [ ]:
W_SPF_H = orig_full[orig_full["Model"].isin(["Wild", "SPF", "Human"])]
W_SPF_H = W_SPF_H[W_SPF_H["Gender"] != "Unknown"]
W_SPF_H_reac = W_SPF_H[rxns]

abundances_per_reac = W_SPF_H_reac.sum(axis=1)
assert len(abundances_per_reac[abundances_per_reac == 0]) == 0